In [7]:
import keras

In [2]:
keras.__version__

'1.2.2'

In [3]:
from googlenet_custom_layers import PoolHelper, LRN
from keras.models import model_from_json

In [8]:
model = model_from_json(open('googlenet_architecture.json').read(),
                       custom_objects={'PoolHelper': PoolHelper, 'LRN': LRN})
model.load_weights('googlenet_weights.h5')

In [9]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 3, 224, 224)   0                                            
____________________________________________________________________________________________________
conv1/7x7_s2 (Convolution2D)     (None, 64, 112, 112)  9472        input_1[0][0]                    
____________________________________________________________________________________________________
zeropadding2d_1 (ZeroPadding2D)  (None, 64, 114, 114)  0           conv1/7x7_s2[0][0]               
____________________________________________________________________________________________________
poolhelper_1 (PoolHelper)        (None, 64, 114, 114)  0           zeropadding2d_1[0][0]            
___________________________________________________________________________________________

In [10]:
??model_from_json

In [12]:
??PoolHelper

In [35]:
from keras.layers import Merge

In [36]:
merge

<function keras.engine.topology.merge>

In [58]:
from keras.models import Sequential
from keras.layers import Dense, merge

left_branch = Sequential()
left_branch.add(Dense(32, input_dim=784))
print(left_branch.output_shape)

right_branch = Sequential()
right_branch.add(Dense(32, input_dim=784))
print(right_branch.output_shape)

merged = Merge([left_branch, right_branch], mode='concat', concat_axis=1)
print(merged.output_shape)

(None, 32)
(None, 32)
(None, 64)


In [60]:
model.output_shape

[(None, 1000), (None, 1000), (None, 1000)]

In [62]:
import cv2
img = imread

ImportError: No module named 'cv2'

In [63]:
model

In [64]:
dir(model)

['__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_fit_loop',
 '_get_node_attribute_at_index',
 '_make_predict_function',
 '_make_test_function',
 '_make_train_function',
 '_output_mask_cache',
 '_output_shape_cache',
 '_output_tensor_cache',
 '_predict_loop',
 '_standardize_user_data',
 '_test_loop',
 '_updated_config',
 'add_inbound_node',
 'add_loss',
 'add_update',
 'add_weight',
 'assert_input_compatibility',
 'build',
 'built',
 'call',
 'compile',
 'compute_mask',
 'constraints',
 'container_nodes',
 'count_params',
 'create_input_layer',
 'evaluate',
 'evaluate_generator',
 'fit',
 'fit_generator',
 'from_config',
 'get_config',
 'get_input_at',
 'get_input_mask_at',
 'g

In [65]:
type(model)

keras.engine.training.Model

In [67]:
model.input_shape

(None, 3, 224, 224)

In [81]:
from PIL import Image
import numpy as np

In [82]:
ls

170518-keras-mnist.ipynb           googlenet_custom_layers.py
170519-lstm-text-generation.ipynb  googlenet_weights.h5
170523-googlenet.ipynb             keras_mlp.py
__pycache__/                       tabby_cat.jpg
googlenet_architecture.json


In [106]:
img = Image.open('tabby_cat.jpg')

In [107]:
img = img.resize((224, 224))

In [108]:
img = np.array(img, dtype='float32')

In [109]:
img.shape

(224, 224, 3)

In [110]:
img[:, :, 0] -= 123.68
img[:, :, 1] -= 116.779
img[:, :, 2] -= 103.939

In [111]:
img[:, :, [0, 1, 2]] = img[:, :, [2, 1, 0]]  # RGB => BGR

In [112]:
img = img.transpose((2, 0, 1))

In [113]:
img.shape

(3, 224, 224)

In [114]:
img = np.expand_dims(img, axis=0)

In [115]:
img.shape

(1, 3, 224, 224)

In [118]:
output = model(img)

In [122]:
output[0]

theano.tensor.var.TensorVariable

In [135]:
import theano
import keras.backend as K

In [131]:
model.input

input_1

In [132]:
model.output

[Softmax.0, Softmax.0, Softmax.0]

In [136]:
f = theano.function(inputs=[model.input, K.learning_phase()], outputs=model.output, allow_input_downcast=True)

In [140]:
K.learning_phase()

keras_learning_phase

In [144]:
result = f(img, 0)

In [149]:
print(result[0].shape)
print(result[1].shape)
print(result[2].shape)

(1, 1000)
(1, 1000)
(1, 1000)


In [150]:
np.argmax(result[0])

282

In [151]:
np.argmax(result[1])

282

In [152]:
np.argmax(result[2])

282

In [155]:
!wget 'https://s3.amazonaws.com/deep-learning-models/image-models/imagenet_class_index.json'

--2017-05-23 15:03:14--  https://s3.amazonaws.com/deep-learning-models/image-models/imagenet_class_index.json
Resolving s3.amazonaws.com... 52.216.17.115
Connecting to s3.amazonaws.com|52.216.17.115|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 35363 (35K) [application/octet-stream]
Saving to: ‘imagenet_class_index.json’

imagenet_class_inde 100%[===================>]  34.53K   107KB/s    in 0.3s    

2017-05-23 15:03:15 (107 KB/s) - ‘imagenet_class_index.json’ saved [35363/35363]



In [157]:
import json
classes = json.load(open('imagenet_class_index.json'))

In [160]:
classes['282']

['n02123159', 'tiger_cat']